## Euclidean distance (experimental)

In [1]:
path = './corpus/a/'

In [2]:
import re

def get_tokens(filename):
    '''open text file and return list of tokens'''
    # text = open(filename, 'r').read().lower()
    f = open(filename, 'r') # open file
    text = f.read() # read file
    text = text.lower() # lower-case text
    tokens = [word for word in re.split('\W', text) if word != ''] # remove punctuation
    return tokens

In [3]:
def get_features(samples):
    tokens = []
    for sample in samples:
        tokens += get_tokens(path + sample + '.txt')
    types = list(set(tokens)) # create unordered list of unique words
    tmp = dict.fromkeys(types, 0) # create temporary dictionary, initialize counts to 0
    for token in tokens: tmp[token] += 1 # count words
    # re-order words in temporary dictionary numerically by descending frequency
    # re-order words with same frequency alphabetically
    features = { 
        key: value for key, value in sorted(tmp.items(),
        key = lambda item: (-item[1], item[0]))
    }
    return features

In [4]:
import pandas as pd

def get_counts(features, samples):
    columns = {}
    for sample in samples:
        columns[sample] = []
        tmp = get_features([sample])
        for feature in features:
            columns[sample].append(tmp.get(feature, 0))
    return pd.DataFrame(columns, index = features)

In [5]:
def get_lengths(samples):
    filenames = [path + sample + '.txt' for sample in samples]
    lengths = {}
    for i in range(len(samples)):
       lengths[samples[i]] = len(get_tokens(filenames[i]))
    return pd.DataFrame(lengths, index = ['words'])

In [6]:
limit = 4 # 4 most frequent words (MFWs)
samples = ['Gratian1', 'dePen', 'Gratian2']
unknown = 'Gratian0'
features = get_features(samples)
mfws = list(features.keys())[:limit]
counts = get_counts(mfws, [unknown] + samples)
lengths = get_lengths([unknown] + samples)
frequencies = (counts / lengths.values) * 1000
csv = open('./CSVs/frequencies_t.csv', 'w')
csv.write(frequencies.transpose().to_csv())
csv.close()
means = frequencies[samples].mean(axis = 1).to_frame('mean')
standard_deviations = frequencies[samples].std(axis = 1).to_frame('std')
z_scores = (frequencies - means.values) / standard_deviations.values
pd.options.display.float_format = '{:,.4f}'.format
z_scores

,Gratian0,Gratian1,dePen,Gratian2
in,-2.8702,-0.4342,-0.7095,1.1437
non,-6.5491,-0.0361,1.0176,-0.9814
et,-3.2375,-0.9786,1.0201,-0.0414
est,-3.5264,0.4179,0.7233,-1.1412


In [7]:
test = z_scores[[unknown]]
test

,Gratian0
in,-2.8702
non,-6.5491
et,-3.2375
est,-3.5264


In [8]:
corpus = z_scores[samples]
corpus

,Gratian1,dePen,Gratian2
in,-0.4342,-0.7095,1.1437
non,-0.0361,1.0176,-0.9814
et,-0.9786,1.0201,-0.0414
est,0.4179,0.7233,-1.1412


In [9]:
differences = (test.values - corpus).abs()
differences

,Gratian1,dePen,Gratian2
in,2.4360,2.1606,4.0139
non,6.5130,7.5667,5.5677
et,2.2589,4.2576,3.1961
est,3.9443,4.2497,2.3852


---

### Manhattan distance

In [10]:
manhattan_row = (differences.mean(axis = 0)).to_frame(unknown).transpose()
manhattan_row

,Gratian1,dePen,Gratian2
Gratian0,3.7880,4.5586,3.7907


### Euclidean distance

Argamon's formula:

$\sqrt{\sum_{i = 1}^n\frac{1}{\sigma_i^2}(f_i(D) - f_i(D'))^2}$

In [11]:
import numpy as np

euclidean_row = np.sqrt(np.square(differences).sum(axis = 0)).to_frame(unknown).transpose()
euclidean_row

,Gratian1,dePen,Gratian2
Gratian0,8.3074,9.9051,7.9382


In [12]:
# equivalent to:
euclidean_distance = np.square(frequencies[samples] - frequencies[[unknown]].values) / np.square(standard_deviations.values)
euclidean_row = np.sqrt(euclidean_distance.sum(axis = 0)).to_frame(unknown).transpose()
euclidean_row

,Gratian1,dePen,Gratian2
Gratian0,8.3074,9.9051,7.9382


### Manhattan distance

In [13]:
limit = 4 # 4 most frequent words (MFWs)
samples = ['Gratian1', 'dePen', 'Gratian2']
unknown = 'psAug'
features = get_features(samples)
mfws = list(features.keys())[:limit]
counts = get_counts(mfws, [unknown] + samples)
lengths = get_lengths([unknown] + samples)
frequencies = (counts / lengths.values) * 1000
means = frequencies[samples].mean(axis = 1).to_frame('mean')
standard_deviations = frequencies[samples].std(axis = 1).to_frame('std')
z_scores = (frequencies - means.values) / standard_deviations.values
test = z_scores[[unknown]]
corpus = z_scores[samples]
differences = (test.values - corpus).abs()
manhattan_row = (differences.mean(axis = 0)).to_frame(unknown).transpose()
euclidean_row = np.sqrt(np.square(differences).sum(axis = 0)).to_frame(unknown).transpose()
manhattan_row

,Gratian1,dePen,Gratian2
psAug,2.6456,1.7373,3.4318


### Euclidean distance

In [14]:
euclidean_row

,Gratian1,dePen,Gratian2
psAug,8.3227,6.1961,8.1540


### Manhattan distance

In [15]:
path = './corpus/a/'

# author candidates, e.g. Gratian 1, the Master of Penance, Gratian 2, etc.
candidates = ['Gratian0', 'Gratian1', 'dePen', 'Gratian2']
manhattan_deltas = pd.DataFrame(columns = candidates)
euclidean_deltas = pd.DataFrame(columns = candidates)
limit = 4 # 4 most frequent words (MFWs)
for candidate in candidates:
    unknown = candidate
    samples = candidates[:]
    samples.remove(unknown)
    features = get_features(samples)
    mfws = list(features.keys())[:limit]
    counts = get_counts(mfws, [unknown] + samples)
    lengths = get_lengths([unknown] + samples)
    frequencies = (counts / lengths.values) * 1000
    means = frequencies[samples].mean(axis = 1).to_frame('mean')
    standard_deviations = frequencies[samples].std(axis = 1).to_frame('std')
    z_scores = (frequencies - means.values) / standard_deviations.values
    test = z_scores[[unknown]]
    corpus = z_scores[samples]
    differences = (test.values - corpus).abs()
    manhattan_row = (differences.mean(axis = 0)).to_frame(unknown).transpose()
    euclidean_row = np.sqrt(np.square(differences).sum(axis = 0)).to_frame(unknown).transpose()
    manhattan_deltas = manhattan_deltas.append(manhattan_row)
    euclidean_deltas = euclidean_deltas.append(euclidean_row)
manhattan_deltas

,Gratian0,Gratian1,dePen,Gratian2
Gratian0,nan,3.7880,4.5586,3.7907
Gratian1,1.4361,nan,0.3628,0.5453
dePen,1.9873,0.4515,nan,0.7673
Gratian2,1.7185,0.6278,0.7905,nan


### Euclidean distance

In [16]:
euclidean_deltas

,Gratian0,Gratian1,dePen,Gratian2
Gratian0,nan,8.3074,9.9051,7.9382
Gratian1,2.9490,nan,0.9617,1.1788
dePen,4.1299,1.2696,nan,1.5687
Gratian2,3.7632,1.4416,1.7420,nan


### Manhattan distance

In [17]:
path = './corpus/a/'

# author candidates, e.g. pseudo-Augustine, Gratian 1, the Master of Penance, Gratian 2, etc.
candidates = ['psAug', 'Gratian1', 'dePen', 'Gratian2']
manhattan_deltas = pd.DataFrame(columns = candidates)
euclidean_deltas = pd.DataFrame(columns = candidates)
limit = 4 # 4 most frequent words (MFWs)
for candidate in candidates:
    unknown = candidate
    samples = candidates[:]
    samples.remove(unknown)
    features = get_features(samples)
    mfws = list(features.keys())[:limit]
    counts = get_counts(mfws, [unknown] + samples)
    lengths = get_lengths([unknown] + samples)
    frequencies = (counts / lengths.values) * 1000
    means = frequencies[samples].mean(axis = 1).to_frame('mean')
    standard_deviations = frequencies[samples].std(axis = 1).to_frame('std')
    z_scores = (frequencies - means.values) / standard_deviations.values
    test = z_scores[[unknown]]
    corpus = z_scores[samples]
    differences = (test.values - corpus).abs()
    manhattan_row = (differences.mean(axis = 0)).to_frame(unknown).transpose()
    euclidean_row = np.sqrt(np.square(differences).sum(axis = 0)).to_frame(unknown).transpose()
    manhattan_deltas = manhattan_deltas.append(manhattan_row)
    euclidean_deltas = euclidean_deltas.append(euclidean_row)
manhattan_deltas

,psAug,Gratian1,dePen,Gratian2
psAug,nan,2.6456,1.7373,3.4318
Gratian1,1.0228,nan,0.4653,0.9325
dePen,0.5178,0.4733,nan,1.3453
Gratian2,5.2005,3.3574,4.2857,nan


### Euclidean distance

In [18]:
euclidean_deltas

,psAug,Gratian1,dePen,Gratian2
psAug,nan,8.3227,6.1961,8.1540
Gratian1,2.4423,nan,1.0498,2.0896
dePen,1.4376,1.0847,nan,2.9794
Gratian2,11.8009,8.5068,10.3475,nan


In [ ]:
path = './corpus/b/'

candidates = ['cases', 'laws', 'orders1', 'orders2', 'simony', 'procedure', 'other1', 'other2', 'monastic', 'other3', 'heresy', 'marriage', 'penance', 'second']
deltas = pd.DataFrame(columns = candidates)
limit = 30 # 30 most frequent words (MFWs)
for candidate in candidates:
    unknown = candidate
    samples = candidates[:]
    samples.remove(unknown)
    features = get_features(samples)
    mfws = list(features.keys())[:limit]
    counts = get_counts(mfws, [unknown] + samples)
    lengths = get_lengths([unknown] + samples)
    frequencies = (counts / lengths.values) * 1000
    means = frequencies[samples].mean(axis = 1).to_frame('mean')
    standard_deviations = frequencies[samples].std(axis = 1).to_frame('std')
    z_scores = (frequencies - means.values) / standard_deviations.values
    test = z_scores[[unknown]]
    corpus = z_scores[samples]
    differences = (test.values - corpus).abs()
    row = np.sqrt(np.square(differences).sum(axis = 0)).to_frame(unknown).transpose()
    deltas = deltas.append(row)
deltas
